<a href="https://colab.research.google.com/github/volpter/ML_comment_selection/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import joblib

In [2]:
import pandas as pd
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, plot_precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
main_df = pd.read_csv('labeled.csv', sep =',' )


In [8]:
main_df.toxic = main_df.toxic.apply(lambda x: int(x))

In [9]:
train_X, test_X, train_y, test_y = train_test_split(main_df.comment, main_df.toxic, random_state=42, test_size=600)

In [10]:
stemmer=SnowballStemmer(language='russian')
def preprocess(comment):
    t_com = word_tokenize(comment, language='russian')
    t_com = [t for t in t_com if t not in string.punctuation ]
    t_com = [t for t in t_com if t not in stopwords.words('russian')]
    t_com = [stemmer.stem(i) for i in t_com]
    return (t_com)
        

In [11]:
train_X = train_X.apply(preprocess)
test_X = test_X.apply(preprocess)

In [26]:
vectorizer = TfidfVectorizer(lowercase=False)
model_pipeline = Pipeline([
                          ('vectorizer', vectorizer),
                          ('model', LogisticRegression(random_state=42))
])

In [27]:
model_pipeline.fit(train_X[:1000], train_y[:1000])

TypeError: ignored

In [20]:
param = [{ #'vectorizer__ngram_range': [(1, 1), (1, 2)],
          #'vectorizer__norm': ('l1', 'l2', None),
          #'vectorizer__max_features': [10, 50, 100, 250, 500, 1000, None],
          'model__solver': ['newton-cg', 'lbfgs', 'sag'],
          'model__penalty': ['none', 'l2']
    
}]
gridsearcher = GridSearchCV(model_pipeline, param, cv=2, verbose=1)

In [21]:
gridsearcher.fit(train_X[:1000], train_y[:1000])

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
12 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", line

AttributeError: ignored

In [9]:
train_set = vectorizer.fit_transform(train_X)

In [10]:
model = LogisticRegression(random_state=42)
model.fit(train_set, train_y)


LogisticRegression(random_state=42)

In [19]:

test_set = vectorizer.transform(test_X)

In [20]:
results=model.predict(test_set)


In [29]:
model_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer = lambda x: preprocess(x))),
    ('model', LogisticRegression(random_state=42))
])

In [32]:
model_pipeline.fit(train_X, train_y)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7f5d33bb6c20>)),
                ('model', LogisticRegression(random_state=42))])

In [36]:
joblib.dump(model_pipeline, 'test.pkl')

PicklingError: ignored